## JUSTICE MATTER stepwise run evaluation
*Experimental Setup Review*
1. Zero Recycling 
* This only means to the simulation run without the MatterUse class
* Emissions Control Rate: RBF Optimized and remains constant.
2. Business as Usual (BAU):
* Recycling Rate: Linear projection based on historical data for each region.
* Emissions Control Rate: RBF Optimized and remains constant.
3. Circular Economy (CE):
* Recycling Rate: All regions achieve 60% by 2050.
* Emissions Control Rate: RBF Optimized and remains constant.

### Zero Recycling
Here relevant variables are analysed for further comparison

In [31]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data
    return array

def collect_variable_data(input_dir, variable_name):
    """
    Collect data for a specific variable from all scenarios.
    """
    scenarios_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            scenarios_data[scenario_name] = data[variable_name]
    return scenarios_data

def plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir):
    """
    Plot a specific variable across different scenarios.
    """
    years = np.arange(2015, 2301)

    fig, axs = plt.subplots(4, 2, figsize=(16, 20))  # 4 rows and 2 columns for 8 subplots
    fig.suptitle(title, fontsize=20)
    
    for ax, (scenario_name, variable_data) in zip(axs.flatten(), scenarios_data.items()):
        mean_data = np.mean(variable_data, axis=0)  # Mean across regions
        for j in range(mean_data.shape[1]):  # Iterate over ensembles
            ax.plot(years, mean_data[:, j])
            
        ax.set_title(scenario_name, fontsize=14)
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Value', fontsize=12)
        ax.grid(True, alpha= 0.3)
    
    plt.tight_layout()
    plt.subplots_adjust(hspace=0.3)

    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{variable_name}.png'), bbox_inches='tight')
    plt.close()

def process_variables_across_scenarios(input_dir, output_dir):
    """
    Process all .npz files in the specified directory and plot variables across scenarios.
    """
    variables_of_interest = [ 'emissions', 'emissions_avoided', 'recycling_cost', 'depletion_ratio']
    titles = ['Global Average Emissions (Gt eq CO2)', 'Global Average Emissions Avoided (Gt eq CO2)', 'Global Average Recycling Cost (T USD)', 'Global Average Depletion Ratio']
    
    for variable_name, title in zip(variables_of_interest, titles):
        scenarios_data = collect_variable_data(input_dir, variable_name)
        plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir)

# Example usage:
input_dir = 'data/output/bau'
output_dir = 'figures/bau'
process_variables_across_scenarios(input_dir, output_dir)


### Circular Economy Case: 60 % recycling rate by 2050

Next cell will produce timeseries plots for each scenario, with the 2050 target aka Circular Economy Scenario 

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data[data.files[0]].item()  # Assuming the file contains a single dictionary
    return array

def plot_scenario(data, scenario_name, output_dir=None):
    """
    Plot the specified variables for a given scenario.
    """
    variables_of_interest = ['depletion_ratio', 'recycling_cost', 'emissions_avoided', 'emissions']
    titles = ['Depletion Ratio', 'Recycling Cost (Trillions)', 'Emissions Avoided (Gt)', 'Emissions (Gt)']
    
    # Create a figure with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'CE Global projections: {scenario_name}', fontsize=16)
    
    years = np.arange(2015, 2301)
    
    for i, var in enumerate(variables_of_interest):
        variable_data = data[var]
        mean_data = np.mean(variable_data, axis=0)  # Mean across regions

        ax = axs[i//2, i%2]
        for j in range(mean_data.shape[0]):  # Iterate over ensembles
            ax.plot(years, mean_data[j,:])  
            
        ax.set_title(titles[i])
        ax.set_xlabel('Year')
        ax.set_ylabel('Value')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the plot
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{scenario_name}.png'))
    plt.close()

def process_all_scenarios(input_dir, output_dir='plots'):
    """
    Process all .npz files in the specified directory.
    """
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            plot_scenario(data, scenario_name, output_dir)

# Example usage:
input_dir = 'data/output/ce'
output_dir = 'figures/ce'
process_all_scenarios(input_dir, output_dir)


Or if we can plot timeseries of the variable across scenarios

In [21]:
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data[data.files[0]].item()  # Assuming the file contains a single dictionary
    return array

def collect_variable_data(input_dir, variable_name):
    """
    Collect data for a specific variable from all scenarios.
    """
    scenarios_data = {}
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            scenarios_data[scenario_name] = data[variable_name]
    return scenarios_data

def plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir):
    """
    Plot a specific variable across different scenarios.
    """
    years = np.arange(2015, 2301)
    colors = sns.color_palette("husl", len(scenarios_data))  # Use a color palette

    fig, axs = plt.subplots(4, 2, figsize=(16, 20))  # 4 rows and 2 columns for 8 subplots
    fig.suptitle(title, fontsize=20)
    
    for ax, (scenario_name, variable_data), color in zip(axs.flatten(), scenarios_data.items(), colors):
        mean_data = np.mean(variable_data, axis=2)  # Mean across ensembles
        for region_index in range(mean_data.shape[0]):  # Iterate over regions
            ax.plot(years, mean_data[region_index, :],  alpha=0.7, color=color)
            
        ax.set_title(scenario_name, fontsize=14)
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Value', fontsize=12)
        ax.grid(True)
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{variable_name}.png'), bbox_inches='tight')
    plt.close()

def process_variables_across_scenarios(input_dir, output_dir):
    """
    Process all .npz files in the specified directory and plot variables across scenarios.
    """
    variables_of_interest = ['depletion_ratio', 'recycling_cost', 'emissions_avoided', 'emissions']
    titles = ['Depletion Ratio', 'Recycling Cost', 'Emissions Avoided (Gt)', 'Emissions (Gt)']
    
    for variable_name, title in zip(variables_of_interest, titles):
        scenarios_data = collect_variable_data(input_dir, variable_name)
        plot_variable_across_scenarios(scenarios_data, variable_name, title, output_dir)

# Example usage:
input_dir = 'data/output/ce'
output_dir = 'figures/ce'
process_variables_across_scenarios(input_dir, output_dir)


BadZipFile: File is not a zip file

### Bussines as Usual Case: Recycling rate linear projections

Next cell will produce timeseries plots for each scenario, Business As Usual scenario

In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt

def load_npz_file(npz_file_path):
    """
    Load a .npz file and return the relevant variables.
    """
    data = np.load(npz_file_path, allow_pickle=True)
    array = data[data.files[0]].item()  # Assuming the file contains a single dictionary
    return array

def plot_scenario(data, scenario_name, output_dir=None):
    """
    Plot the specified variables for a given scenario.
    """
    variables_of_interest = ['depletion_ratio', 'recycling_cost', 'emissions_avoided', 'emissions']
    titles = ['Depletion Ratio', 'Recycling Cost (Trillions)', 'Emissions Avoided (Gt)', 'Emissions (Gt)']
    
    # Create a figure with 2 rows and 2 columns
    fig, axs = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle(f'CE Global projections: {scenario_name}', fontsize=16)
    
    years = np.arange(2015, 2301)
    
    for i, var in enumerate(variables_of_interest):
        variable_data = data[var]
        mean_data = np.mean(variable_data, axis=2)  # Mean across regions

        ax = axs[i//2, i%2]
        for j in range(mean_data.shape[0]):  # Iterate over ensembles
            ax.plot(years, mean_data[j,:])  
            
        ax.set_title(titles[i])
        ax.set_xlabel('Year')
        ax.set_ylabel('Value')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the plot
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(os.path.join(output_dir, f'{scenario_name}.png'))
    plt.close()

def process_all_scenarios(input_dir, output_dir='plots'):
    """
    Process all .npz files in the specified directory.
    """
    for file_name in os.listdir(input_dir):
        if file_name.endswith('.npz'):
            npz_file_path = os.path.join(input_dir, file_name)
            scenario_name = os.path.splitext(file_name)[0]
            data = load_npz_file(npz_file_path)
            plot_scenario(data, scenario_name, output_dir)

# Example usage:
input_dir = 'data/output/bau'
output_dir = 'figures/bau'
process_all_scenarios(input_dir, output_dir)
